In [2]:
%pip install -q otter-grader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 20.8 MB/s eta 0:00:00


In [3]:
!mkdir tests

In [4]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS 595 Homework 1: Merkle Trees and Cryptographic Hashing

**Due date**: Tuesday, September 16 at 11:59pm on [Gradescope](https://www.gradescope.com/courses/1118328) (entry code 2244N4).

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment consists of three parts:
1. Verify that a given Merkle Tree root has been computed correctly.
2. Verify that the transaction is included in the block by computing the Merkle Tree path to the root.
3. Modify the hash function to use SHA-256 truncated to the first 12 bits and demonstrate how to forge transaction inclusion.

To begin, first load the autograder tests in one of the following ways:
- If you are using Google Colab, execute the code block below and upload the zipped tests folder
- If you are running Jupyter notebook locally, unzip tests.zip and ensure the tests folder is in the same directory as this notebook. Then continue to part 1.


In [5]:
# Upload and extract the tests directory (only necessary in Colab)
from google.colab import files
uploaded = files.upload()

import zipfile
with zipfile.ZipFile("tests.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

Saving tests.zip to tests.zip


**PART 1: Verifying the Merkle Tree Root**

In this part, you will implement a function `compute_merkle_root` that computes the Merkle Tree root from a list of transactions. Recall the structure of a Merkle tree: Each transaction in the list is hashed, and that hash is considered a leaf in a binary tree. The Merkle root is computed as follows
- Concatenate the hashes of adjacent leaves together and hash them to produce a value for the parent node.
- Recursively repeat, adding a new layer of hashes each time, until you reach a single top-level hash -- this value is the Merkle root.

For the hash function, use the sha256() function provided in the cell below. When computing the Merkle root, you may assume that the length of the provided transactions list is a power of two.

In [6]:
import hashlib
def sha256(input_data: str) -> str:
    """
    Compute a SHA-256 hash

    Args:
        input_data (str): Input data to hash.

    Returns:
        str: 256 bit hex string
    """
    return hashlib.sha256(input_data.encode('utf-8')).hexdigest()

In [8]:
def compute_merkle_root(transactions: list[str], hash_function: callable=sha256) -> str:
    """
    Args:
        transactions (list of str): A list of transaction strings.
        hash_function: the hash function to use.

    Returns:
        str: The computed Merkle Tree root as a hexadecimal string.
    """
    # TODO: implement this function

    #if there's no any transactions, directly return None
    if not transactions:
        return None

    # init leaf: hash on every transaction
    level = [hash_function(tx) for tx in transactions]

    # merge layer by layer unmtil there's only one root left
    while len(level) > 1:
        new_level = []
        for i in range(0, len(level), 2):
            left = level[i]
            right = level[i] if i + 1 == len(level) else level[i + 1]
            combined = bytes.fromhex(left) + bytes.fromhex(right)
            new_level.append(hash_function(combined.hex()))
        level = new_level

    return level[0]

As you will see throughout this course, crypto code can be incredibly precise and finicky. For this reason: most programming questions will be accommpanied by **test cases**, which are example inputs and outputs to a function that allow you to check whether your code is working properly.

In this question, we provide test cases in human-readable format in the comment above, and we also provide unit tests for you to use below.

_Points:_ 30

In [9]:
transactions = ["tx1", "tx2", "tx3", "tx4", "tx5", "tx6", "tx7", "tx8"]
expected_root = "ca12ad3f60dde6fcadc46c58f7a877648c9119484bd2e3178a1b65da78111efa"
computed_root = compute_merkle_root(transactions)
computed_root == expected_root

True

In [10]:
transactions = ['BawAgfmCqNXdsB5v', 'G5ZDcwuzdq2V28Or', 'yg1B31IWCRVp2kv9', 'kRWhaW4d3B4y2Ro5', '0yGvlL3QLIAzmxA3', 'P1eiNLr7pyhmJAGD', 'gds3vD1o3cCaPsiu', 'vIPKj6cDlduPbs6o']
expected_root = "23a979478ee196e22ec7b488e088e6c1b5e8ab8f7f472326418091cf81d8762c"
computed_root = compute_merkle_root(transactions)
computed_root == expected_root

True

In [11]:
grader.check("q1")

q1 results: All test cases passed!

**PART 2: Verifying Transaction Inclusion:**

In this part of the homework, you will implement a function `gen_transaction_path` that computes the Merkle Tree path for a specific transaction.
- Start with the hash of the transaction at the given index.
- Compute the sibling hash and whether it's on the left or right at each level.

You are provided with a `verify_path` function, which allows you to verify whether a transaction is included in the Merkle Tree by recomputing the root.

You can use this function to test whether your implementation of `gen_transaction_path` is correct.

In [12]:
def verify_path(merkle_root: str, merkle_path: list[tuple[str, str]], transaction: str, hash_function: callable=sha256) -> bool:
    """
    Verify whether a transaction is included in the Merkle Tree by recomputing the root.

    Args:
        merkle_root (str): The Merkle Tree root to verify against.
        merkle_path (list of tuple): The Merkle Tree path to the root.
                                     Each tuple contains:
                                        - The sibling hash (str)
                                        - A string indicating if the sibling is "left" or "right".
        transaction (str): The transaction to verify.
        hash_function: the hash function to use.

    Returns:
        bool: True if the transaction is included in the Merkle Tree, False otherwise.
    """


    # Start with the hash of the transaction
    current_hash = hash_function(transaction)

    # Traverse the path and recompute the hashes up to the root
    for sibling_hash, position in merkle_path:
        if position == "left":
            current_hash = hash_function((sibling_hash + current_hash))
        elif position == "right":
            current_hash = hash_function((current_hash + sibling_hash))
        else:
            raise ValueError("Invalid position in the Merkle path: must be 'left' or 'right'.")

    # Check if the recomputed root matches the provided Merkle root
    return current_hash == merkle_root



In [19]:
def gen_transaction_path(transactions: list[str], index: int, hash_function: callable=sha256) -> list[tuple[str, str]]:
    """
    Args:
        transactions (list of str): A list of transaction strings.
        index (int): The zero-based index of the transaction to verify.
        hash_function: the hash function to use.

    Returns:
        list of tuple: A list of tuples representing the Merkle Tree path.
        Each tuple contains:
            - The sibling hash (str)
            - A string indicating if the sibling is "left" or "right".
    """
    # TODO: implement this function

    # init leaf node: hash on every transaction
    level = [hash_function(tx) for tx in transactions]
    path = []
    idx = index

    # from bottom to up path
    while len(level) > 1:
        # if it's the last and odd, copy itself
        if idx % 2 == 0:  # current node is left node
            sibling_index = idx + 1 if idx + 1 < len(level) else idx
            sibling_pos = "right"
        else:             # current node is right node
            sibling_index = idx - 1
            sibling_pos = "left"

        if sibling_index != idx:
            path.append((level[sibling_index], sibling_pos))

        #
        new_level = []
        for i in range(0, len(level), 2):
            left = level[i]
            right = level[i] if i + 1 == len(level) else level[i + 1]
            combined = bytes.fromhex(left) + bytes.fromhex(right)
            new_level.append(hash_function(combined.hex()))
        level = new_level
        idx //= 2

    return path

Once again, we provide several unit tests for you.

_Points:_ 30

In [ ]:
grader.check("q2")

q2 results: All test cases passed!

**PART 3: Forging Inclusion with Truncated Hashes:**

In this section, you need to modify the hash function to use a truncated SHA-256 hash (keeping only the first 12 bits, i.e. first three chars of the hex representation) and demonstrate how to forge transaction inclusion.

Write a function `forge_transaction_path` that takes the following inputs:

merkle_root: The Merkle tree root hash.
false_transaction: The specific transaction to forge.
The function should leverage the truncated SHA-256 hash to forge a transaction inclusion proof for the specified transaction.

In [20]:
def truncated_sha256(input_data: str) -> str:
    """
    Compute a SHA-256 hash truncated to the first 12 bits (first 3 chars of the hex representation).

    Args:
        input_data (str): Input data to hash.

    Returns:
        str: The truncated hash as a hex string (first 12 bits).
    """
    full_hash = hashlib.sha256(input_data.encode('utf-8')).hexdigest()
    return full_hash[:3]

In [17]:
def forge_transaction_path(merkle_root: str, false_transaction: str) -> list[tuple[str, str]]:
    """
    Demonstrate how to forge transaction inclusion using a truncated hash function
    for a specific transaction.

    Args:
        merkle_root (str): The actual Merkle Tree root to verify against.
        false_transaction (str): The transaction to forge inclusion for.

    Returns:
        list[tuple[str, str]]: A forged Merkle tree path from the false transaction (at any index) to the real Merkle root. Each tuple contains:
            - The sibling hash (str)
            - A string indicating if the sibling is "left" or "right".
    """

# TODO: implement the rest of this function

    # compute fake transaction truncated hash
    start_hash = truncated_sha256(false_transaction)

    # We want to find a sibling such that truncated_sha256(start || sibling) == merkle_root
    # or truncated_sha256(sibling || start) == merkle_root.
    # Since only 3 hex chars are needed, brute force is fast.
    for candidate in range(10000):
        fake_sibling = hex(candidate)[2:]  # use hex string as sibling
        # try as right sibling
        if truncated_sha256(start_hash + fake_sibling) == merkle_root:
            return [(fake_sibling, "right")]
        # try as left sibling
        if truncated_sha256(fake_sibling + start_hash) == merkle_root:
            return [(fake_sibling, "left")]

    # if not found, return empty path (fallback)
    return []

We have provided two public tests, but you should try to run this function on many inputs, and observe the resulting pattern.

_Points:_ 30

In [18]:
grader.check("q3")

q3 results: All test cases passed!

_Points:_ 10

We've shown in class and lab that Merkle trees, when properly designed and used, are computationally sound. That is, it is exceedingly difficult to produce a Merkle path for a false transaction that successfully verifies against a real root. What went wrong in this problem, and how did you exploit it to create forged paths?



--- Write your answer here ---



## Submitting the Assignment

Congratulations on completing the first assignment! Here's how to submit it and receive credit.

**Documenting collaborators, sources, and AI tools:** In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. Any external code or tools used that you used. If you used code or assistance from external sources, including AI tools, you must provide details about how you used them. For AI tools, <span style="background-color: yellow;">**include the exact prompts or screenshots of your interaction**</span> to demonstrate how you utilized the tool effectively to achieve your results. This will help us understand your process and ensure proper use of resources..

Your response:

1. I complete all parts by myself.

2.

3. my prompt: I’m working on a homework problem about forging inclusion proofs with truncated SHA-256 hashes in a Merkle tree. Can you explain how I might approach finding a sibling hash that matches a given truncated root?

**Sending to Gradescope:**

After completing the assignment:
    if you did the assignment on Colab, download it in `.ipynb` format
    if you did the assignment locally on your machine, all you need to do is to find it in your directory

Then, submit only the `.ipynb` file to Coding Assignment 1 on the GradeScope and it takes a while for the auto grading system to check your work.